# 余因子と余因子展開

## 概要
この記事では、行列の基本的な概念である余因子と余因子展開について解説する。これらの概念は、線形代数学や機械学習、特に推薦システムにおいて重要な役割を果たす。具体例を通じて、Pythonによる実装例も示す。


### github
- jupyter notebook形式のファイルは[こちら](https://github.com/hiroshi0530/wa-src/blob/master/rec/linalg/05/05_nb.ipynb)

### google colaboratory
- google colaboratory で実行する場合は[こちら](https://colab.research.google.com/github/hiroshi0530/wa-src/blob/master/rec/linalg/05/05_nb.ipynb)


### 実行環境
OSはmacOSです。LinuxやUnixのコマンドとはオプションが異なりますので注意してください。

In [18]:
!sw_vers

ProductName:		macOS
ProductVersion:		13.5.1
BuildVersion:		22G90


In [19]:
!python -V

Python 3.9.17


pandasのテーブルを見やすいようにHTMLのテーブルにCSSの設定を行います。

In [20]:
from IPython.core.display import HTML

style = """
<style>
    .dataframe thead tr:only-child th {
        text-align: right;
    }

    .dataframe thead th {
        text-align: left;
        padding: 5px;
    }

    .dataframe tbody tr th {
        vertical-align: top;
        padding: 5px;
    }

    .dataframe tbody tr:hover {
        background-color: #ffff99;
    }

    .dataframe {
        background-color: white;
        color: black;
        font-size: 16px;
    }

</style>
"""
HTML(style)

基本的なライブラリをインポートし watermark を利用してそのバージョンを確認しておきます。
ついでに乱数のseedの設定をします。

In [21]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

In [22]:
import random

import scipy
import numpy as np

import matplotlib
import matplotlib.pyplot as plt

seed = 123
random_state = 123

random.seed(seed)
np.random.seed(seed)


from watermark import watermark

print(watermark(python=True, watermark=True, iversions=True, globals_=globals()))

Python implementation: CPython
Python version       : 3.9.17
IPython version      : 8.17.2

matplotlib: 3.8.1
numpy     : 1.25.2
scipy     : 1.11.2

Watermark: 2.4.3



## 余因子の定義と性質

余因子（Cofactor）とは、行列の要素に関連するある種の行列式を指す。具体的には、$a_{ij}$を行列$\mathbf{A}$の$(i,j)$成分としたとき、余因子$C_{ij}$は次のように定義される：

$$
C_{ij} = (-1)^{i+j} \det(\mathbf{A}_{ij})
$$

ここで、$\mathbf{A}_{ij}$は$\mathbf{A}$から$i$行と$j$列を取り除いた部分行列である。この定義により、余因子行列$\mathbf{C}$は行列$\mathbf{A}$の各要素の余因子を集めた行列となる。

### 具体例
例えば、行列$\mathbf{A}$が次のような$3 \times 3$行列であるとする：

$$
\mathbf{A} = \begin{pmatrix}
1 & 2 & 3 \\
0 & 1 & 4 \\
5 & 6 & 0
\end{pmatrix}
$$

このとき、成分$a_{11}$に対応する余因子$C_{11}$は次のように計算される：

$$
\mathbf{A}_{11} = \begin{pmatrix}
1 & 4 \\
6 & 0
\end{pmatrix}
$$

$$
C_{11} = (-1)^{1+1} \det(\mathbf{A}_{11}) = \det(\begin{pmatrix}
1 & 4 \\
6 & 0
\end{pmatrix}) = 1 \cdot 0 - 4 \cdot 6 = -24
$$

### Pythonによる実装例



In [23]:
import numpy as np


def minor(matrix, i, j):
    minor_matrix = np.delete(matrix, i, axis=0)
    minor_matrix = np.delete(minor_matrix, j, axis=1)
    return minor_matrix


def cofactor(matrix, i, j):
    minor_matrix = minor(matrix, i, j)
    return ((-1) ** (i + j)) * np.linalg.det(minor_matrix)


A = np.array([[1, 2, 3], [0, 1, 4], [5, 6, 0]])

C11 = cofactor(A, 0, 0)
print("C11 =", round(C11, 2))

C11 = -24.0




## 余因子展開の定義と性質

余因子展開（Cofactor Expansion）とは、行列の行や列に沿って行う行列式の計算方法である。行列$\mathbf{A}$の行列式$\det(\mathbf{A})$は、次のように余因子を用いて展開される：

$$
\det(\mathbf{A}) = \sum_{j=1}^{n} a_{ij} C_{ij}
$$

ここで、$i$は固定された行のインデックスであり、$j$はその行の各列のインデックスである。この式は任意の行$i$について成立する。

### 具体例

再び行列$\mathbf{A}$を用いて、行列式を余因子展開により計算する。$i=1$行に沿って展開すると：

$$
\det(\mathbf{A}) = a_{11}C_{11} + a_{12}C_{12} + a_{13}C_{13}
$$

それぞれの余因子は以下の通り計算される：

$$
C_{11} = -24, \quad C_{12} = (-1)^{1+2} \det(\begin{pmatrix}
0 & 4 \\
5 & 0
\end{pmatrix}) = 20, \quad C_{13} = (-1)^{1+3} \det(\begin{pmatrix}
0 & 1 \\
5 & 6
\end{pmatrix}) = -5
$$

したがって、

$$
\det(\mathbf{A}) = 1 \cdot (-24) + 2 \cdot 20 + 3 \cdot (-5) = -24 + 40 - 15 = 1
$$

### Pythonによる実装例



In [24]:
def determinant(matrix):
    n = len(matrix)
    if n == 1:
        return matrix[0, 0]
    elif n == 2:
        return matrix[0, 0] * matrix[1, 1] - matrix[0, 1] * matrix[1, 0]
    else:
        det = 0
        for j in range(n):
            det += matrix[0, j] * cofactor(matrix, 0, j)
        return det


det_A = determinant(A)
print("det(A) =", round(det_A, 2))

det(A) = 1.0




## 余因子と余因子展開の応用

余因子と余因子展開は、行列の逆行列の計算にも重要である。特に、行列$\mathbf{A}$の逆行列$\mathbf{A}^{-1}$は、余因子行列$\mathbf{C}$と行列式$\det(\mathbf{A})$を用いて次のように表される：

$$
\mathbf{A}^{-1} = \frac{1}{\det(\mathbf{A})} \mathbf{C}^T
$$

この関係は、行列の性質を解析する際に有用である。特に機械学習における線形代数の応用、例えば推薦システムのモデルの構築において重要である。

### 具体例

行列$\mathbf{A}$の逆行列を求めるために、余因子行列と行列式を利用する。

### Pythonによる実装例



In [25]:
def cofactor_matrix(matrix):
    n = len(matrix)
    C = np.zeros(matrix.shape)
    for i in range(n):
        for j in range(n):
            C[i, j] = cofactor(matrix, i, j)
    return C


C = cofactor_matrix(A)
det_A = determinant(A)
A_inv = (1 / det_A) * C.T
print("A^(-1) = \n", A_inv.round(2))

A^(-1) = 
 [[-24.  18.   5.]
 [ 20. -15.  -4.]
 [ -5.   4.   1.]]



## 結論

この記事では、余因子と余因子展開の定義、性質、及び具体的な応用例について説明した。これらの概念は、行列の性質を解析するための強力なツールであり、特に機械学習の分野において重要である。

### 参考文献
- Wikipedia: [Cofactor (linear algebra)](https://en.wikipedia.org/wiki/Cofactor_(linear_algebra))
- NumPy Documentation: [numpy.linalg.det](https://numpy.org/doc/stable/reference/generated/numpy.linalg.det.html)
- Scipy Documentation: [scipy.linalg.inv](https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.inv.html)